In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
from collections import Counter

In [2]:
# Inicializando o MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Pasta com os vídeos
pasta_videos = './pasta2 - Copia'

# Listas para armazenar os dados e os rótulos
dados_uma_mao = []
rotulos_uma_mao = []
dados_duas_maos = []
rotulos_duas_maos = []

# Dicionário para armazenar as coordenadas por palavra
coordenadas_por_palavra = {'uma': {}, 'duas': {}}

In [3]:
for categoria in ['uma', 'duas']:
    caminho_categoria = os.path.join(pasta_videos, categoria)
    for subpasta in os.listdir(caminho_categoria):
        caminho_subpasta = os.path.join(caminho_categoria, subpasta)
        for video in os.listdir(caminho_subpasta):
            # Lendo o vídeo
            cap = cv2.VideoCapture(os.path.join(caminho_subpasta, video))
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Convertendo a cor da imagem
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Processando a imagem
                result = hands.process(image)
                
                if result.multi_hand_landmarks and result.multi_handedness:
                    if categoria == 'uma':
                        # Para palavras de uma mão, consideramos apenas a mão direita
                        for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
                            hand_label = handedness.classification[0].label.lower()
                            if hand_label == 'right':
                                # Extraindo as coordenadas
                                coords = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten())
                                
                                # Adicionando as coordenadas à lista de dados
                                dados_uma_mao.append(coords)
                                # Adicionando o rótulo à lista de rótulos
                                rotulos_uma_mao.append(subpasta)
                                
                                # Armazenando as coordenadas por palavra
                                if subpasta not in coordenadas_por_palavra['uma']:
                                    coordenadas_por_palavra['uma'][subpasta] = []
                                coordenadas_por_palavra['uma'][subpasta].append(tuple(coords))
                    else:
                        # Para palavras de duas mãos, consideramos as duas mãos
                        for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
                            # Identificar se a mão é direita ou esquerda
                            hand_label = handedness.classification[0].label.lower()
                            if hand_label == 'right':
                                hand_label = 'direita'
                            else:
                                hand_label = 'esquerda'
                            
                            # Extraindo as coordenadas
                            coords = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten())
                            
                            # Adicionando as coordenadas à lista de dados
                            dados_duas_maos.append(coords)
                            # Adicionando o rótulo à lista de rótulos
                            rotulos_duas_maos.append(subpasta)
                            
                            # Armazenando as coordenadas por palavra e mão
                            if subpasta not in coordenadas_por_palavra['duas']:
                                coordenadas_por_palavra['duas'][subpasta] = []
                            coordenadas_por_palavra['duas'][subpasta].append(tuple(coords))

# Selecionando a coordenada mais frequente por palavra
coordenadas_selecionadas_uma = {}
coordenadas_selecionadas_duas = {'direita': {}, 'esquerda': {}}

for palavra, coords_lista in coordenadas_por_palavra['uma'].items():
    # Contar a frequência de cada conjunto de coordenadas
    contador = Counter(coords_lista)
    # Selecionar a coordenada mais frequente
    mais_frequente = contador.most_common(1)[0][0]
    coordenadas_selecionadas_uma[palavra] = mais_frequente

for hand_label in ['direita', 'esquerda']:
    for palavra, coords_lista in coordenadas_por_palavra['duas'].items():
        # Contar a frequência de cada conjunto de coordenadas
        contador = Counter(coords_lista)
        # Selecionar a coordenada mais frequente
        mais_frequente = contador.most_common(1)[0][0]
        coordenadas_selecionadas_duas[hand_label][palavra] = mais_frequente

# Preparando os dados para treinamento
dados_filtrados_uma = []
rotulos_filtrados_uma = []
dados_filtrados_duas = []
rotulos_filtrados_duas = []

for palavra, coords in coordenadas_selecionadas_uma.items():
    dados_filtrados_uma.append(coords)
    rotulos_filtrados_uma.append(palavra)

for hand_label in ['direita', 'esquerda']:
    for palavra, coords in coordenadas_selecionadas_duas[hand_label].items():
        dados_filtrados_duas.append(coords)
        rotulos_filtrados_duas.append(palavra)

# Convertendo as listas em arrays numpy
dados_uma_mao = np.array(dados_filtrados_uma)
rotulos_uma_mao = np.array(rotulos_filtrados_uma)
dados_duas_maos = np.array(dados_filtrados_duas)
rotulos_duas_maos = np.array(rotulos_filtrados_duas)

# Balanceamento dos dados
min_amostras_uma = min(Counter(rotulos_uma_mao).values())
dados_balanceados_uma = []
rotulos_balanceados_uma = []
for palavra in coordenadas_selecionadas_uma.keys():
    indices = [i for i, r in enumerate(rotulos_uma_mao) if r == palavra]
    indices_selecionados = np.random.choice(indices, min_amostras_uma, replace=False)
    dados_balanceados_uma.extend(dados_uma_mao[indices_selecionados])
    rotulos_balanceados_uma.extend(rotulos_uma_mao[indices_selecionados])

dados_balanceados_uma = np.array(dados_balanceados_uma)
rotulos_balanceados_uma = np.array(rotulos_balanceados_uma)

min_amostras_duas = min(Counter(rotulos_duas_maos).values())
dados_balanceados_duas = []
rotulos_balanceados_duas = []
for palavra in coordenadas_selecionadas_duas['direita'].keys():
    indices = [i for i, r in enumerate(rotulos_duas_maos) if r == palavra]
    indices_selecionados = np.random.choice(indices, min_amostras_duas, replace=False)
    dados_balanceados_duas.extend(dados_duas_maos[indices_selecionados])
    rotulos_balanceados_duas.extend(rotulos_duas_maos[indices_selecionados])

dados_balanceados_duas = np.array(dados_balanceados_duas)
rotulos_balanceados_duas = np.array(rotulos_balanceados_duas)

# Dividindo os dados em conjuntos de treinamento e teste
X_train_uma, X_test_uma, y_train_uma, y_test_uma = train_test_split(dados_balanceados_uma, rotulos_balanceados_uma, test_size=0.2, random_state=42)
X_train_duas, X_test_duas, y_train_duas, y_test_duas = train_test_split(dados_balanceados_duas, rotulos_balanceados_duas, test_size=0.2, random_state=42)

# Treinando os modelos com as coordenadas selecionadas
modelo_uma = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
modelo_uma.fit(X_train_uma, y_train_uma)

modelo_duas = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
modelo_duas.fit(X_train_duas, y_train_duas)

# Avaliando os modelos
score_uma = modelo_uma.score(X_test_uma, y_test_uma)
score_duas = modelo_duas.score(X_test_duas, y_test_duas)
print(f'Acurácia do modelo (uma mão): {score_uma*100:.2f}%')
print(f'Acurácia do modelo (duas mãos): {score_duas*100:.2f}%')

# Salvando os modelos
joblib.dump(modelo_uma, 'modelo_uma_mao.pkl')
joblib.dump(modelo_duas, 'modelo_duas_maos.pkl')

c:\Users\danil\Documents\GitHub\TCC2_modelo_IA\myenv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Acurácia do modelo (uma mão): 0.00%
Acurácia do modelo (duas mãos): 0.00%


['modelo_duas_maos.pkl']